In [ ]:
!pip install sklearn-genetic-opt

In [1]:
import subprocess
import numpy as np
import os

from sklearn.base import BaseEstimator, RegressorMixin

# Random Search one parameter optimization
Example, make sure it works before going to GA.  

Score(fitness function) computes with MSE loss and PSNR like:  
$$score = 1-MSE + PSNR/100$$

In [ ]:
class FDNeRF(BaseEstimator, RegressorMixin):  
    def __init__(self, n_layers=0):
        # Parameters should have same name as attributes
        self.n_layers = n_layers

    def fit(self, X, y=None):
        command = 'python run_nerf.py --config configs/lego.txt --n_iters 100'
        command += ' --n_layers '+ str(self.n_layers)
        command += ' --n_layers_fine '+str(self.n_layers)
        command += ' --i_print '+str(20)
        
        print(command)
        p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)

        (output, err) = p.communicate()  

        #This makes the wait possible
        p_status = p.wait()
        # print('Training done')

        return self
    
    def predict(self, X, y=None):
        try:
            getattr(self, "treshold_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")

        return self.result

    def score(self, X, y=None):
        train_dirs = sorted(os.listdir('logs'))
        train_dirs.remove('summaries')
        train_dirs_split = [x.split(' ')[1] for x in train_dirs]
        inexies = [i[0] for i in sorted(enumerate(train_dirs_split), key=lambda x:x[1])]
        mask = np.array(inexies) == max(inexies)
        file_name = np.array(train_dirs)[mask]

        last_train_file = 'logs/' + file_name[0] + '/loss_vs_time.pkl'

        data = np.load(last_train_file, allow_pickle=True)
        loss,psnr,time = data['losses'][-1], data['psnr'][-1], data['time'][-1]
        score = 1-loss + psnr/100
        print(f'score: {score}')
        return score

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
N = 3
X_train = np.linspace(1, 3, num=N)
X_test = np.linspace(1, 3, num=N)
dummy_y = [1 for i in range(N)]
tuned_params = {
    "n_layers" : np.array([1, 2, 8]).astype(int)
    }


gs = GridSearchCV(FDNeRF(), tuned_params, cv=[(slice(None), slice(None))])

# for some reason I have to pass y with same shape
# otherwise gridsearch throws an error. Not sure why.
# X and Y dosn't mean anything
# just makecompatable with sklearn
gs.fit(X_test, y=dummy_y)

gs.best_params_


# Genetic Optimization 
Score(fitness function) computes with MSE loss and PSNR like:  
$$t = time.reverse()$$
$$score = 10*\sum{(1-MSE)*t} +  \sum{PSNR*t}$$
Intuition behid that is, the longer training going the more insignificant past measurements become.  
Just like weighted sum.

In [2]:
class FDNeRF_GA(BaseEstimator, RegressorMixin):  
    def __init__(self, 
                 n_layers=0,
                 n_width=0,
                 n_layers_fine=0,
                 n_width_fine=0,
                 n_layers_time=0,
                 n_width_time=0,
                 multires=0,
                 multires_timenet=0,
                 ):
        # Parameters should have same name as attributes
        self.n_layers=n_layers
        self.n_width=n_width
        self.n_layers_fine=n_layers_fine
        self.n_width_fine=n_width_fine
        self.n_layers_time=n_layers_time
        self.n_width_time=n_width_time
        self.multires=multires
        self.multires_timenet=multires_timenet


    def fit(self, X, y=None):
        command = 'python run_nerf.py --config configs/lego.txt --n_iters 100'
        command += ' --i_print '+str(20)
        
        command += ' --n_layers '+ str(self.n_layers)
        command += ' --n_width '+str(self.n_width)
        
        command += ' --n_layers_fine '+str(self.n_layers_fine)
        command += ' --n_width_fine '+str(self.n_width_fine)

        command += ' --n_layers_time '+str(self.n_layers_time)
        command += ' --n_width_time '+str(self.n_width_time)
        
        
        print(command)
        p = subprocess.Popen(command,stdout=subprocess.PIPE, shell=True)

        (output, err) = p.communicate()  
        if err:
            print('PIPE ERROR: ', err)
        #This makes the wait possible
        p_status = p.wait()

        return self
    
    def predict(self, X, y=None):
        try:
            getattr(self, "treshold_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")

        return self.result

    def _compute_score(self,data):
        psnr = np.array(data['psnr']) 
        n = psnr.shape[0]
        rev_time = np.linspace(1,n)[::-1]
        _psnr = (psnr / rev_time).sum()

        loss = (1 - np.array(data['losses'])) / rev_time
        _loss = loss.sum()*10
        score =_psnr + _loss
        return score


    def score(self, X, y=None):
        train_dirs = sorted(os.listdir('logs'))
        if 'summaries' in train_dirs:
            train_dirs.remove('summaries')
        train_dirs_split = [x.split(' ')[1]+x.split(' ')[2] for x in train_dirs]
        inexies = [i[0] for i in sorted(enumerate(train_dirs_split), key=lambda x:x[1])]
        mask = np.array(inexies) == max(inexies)
        file_name = np.array(train_dirs)[mask]

        last_train_file = 'logs/' + file_name[0] + '/loss_vs_time.pkl'
        if os.path.isfile(last_train_file):
            data = np.load(last_train_file, allow_pickle=True)
            # loss,psnr,time = data['losses'][-1], data['psnr'][-1], data['time'][-1]
            # score = 1-loss + psnr/100
            score = self._compute_score(data)
            print(f'score: {score}')
            return score
        return -1 

array([6.        , 5.89795918, 5.79591837, 5.69387755, 5.59183673,
       5.48979592, 5.3877551 , 5.28571429, 5.18367347, 5.08163265,
       4.97959184, 4.87755102, 4.7755102 , 4.67346939, 4.57142857,
       4.46938776, 4.36734694, 4.26530612, 4.16326531, 4.06122449,
       3.95918367, 3.85714286, 3.75510204, 3.65306122, 3.55102041,
       3.44897959, 3.34693878, 3.24489796, 3.14285714, 3.04081633,
       2.93877551, 2.83673469, 2.73469388, 2.63265306, 2.53061224,
       2.42857143, 2.32653061, 2.2244898 , 2.12244898, 2.02040816,
       1.91836735, 1.81632653, 1.71428571, 1.6122449 , 1.51020408,
       1.40816327, 1.30612245, 1.20408163, 1.10204082, 1.        ])

In [3]:
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Categorical, Continuous,Integer
# https://sklearn-genetic-opt.readthedocs.io/en/stable/api/space.html
param_grid = {
    "n_layers" : Integer(1,8),
    "n_width": Integer(64,256),
    "n_layers_fine" : Integer(1,8),
    "n_width_fine": Integer(64,256),
    "n_layers_time": Integer(1,8),
    "n_width_time": Integer(64,256),
    }


evolved_estimator = GASearchCV(FDNeRF_GA(),
                    cv=[(slice(None), slice(None))],
                    # scoring='accuracy',
                    param_grid=param_grid,
                    population_size=10,
                    generations=8,
                    tournament_size=3,
                    elitism=True,
                    verbose=True)

In [4]:
N = 3
dummy_x = np.linspace(1, 3, num=N)
dummy_y = [1 for i in range(N)]

evolved_results = evolved_estimator.fit(dummy_x,dummy_y)

python run_nerf.py --config configs/lego.txt --n_iters 100 --i_print 20 --n_layers 8 --n_width 92 --n_layers_fine 3 --n_width_fine 249 --n_layers_time 6 --n_width_time 158
score: 9.272331865897725
python run_nerf.py --config configs/lego.txt --n_iters 100 --i_print 20 --n_layers 4 --n_width 106 --n_layers_fine 5 --n_width_fine 203 --n_layers_time 3 --n_width_time 156
score: 8.627637253675974
python run_nerf.py --config configs/lego.txt --n_iters 100 --i_print 20 --n_layers 2 --n_width 250 --n_layers_fine 4 --n_width_fine 172 --n_layers_time 3 --n_width_time 185
score: 9.5690137285903
python run_nerf.py --config configs/lego.txt --n_iters 100 --i_print 20 --n_layers 5 --n_width 88 --n_layers_fine 5 --n_width_fine 226 --n_layers_time 7 --n_width_time 154
score: 7.67982669729715
python run_nerf.py --config configs/lego.txt --n_iters 100 --i_print 20 --n_layers 2 --n_width 134 --n_layers_fine 1 --n_width_fine 114 --n_layers_time 6 --n_width_time 131
score: 8.282964239355552
python run_nerf

KeyboardInterrupt: 

In [ ]:
evolved_estimator.best_params